Insper

# Aula 6 - DataFrames

In [1]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("Michel_DF") \
            .getOrCreate()

In [2]:
spark

In [3]:
sc = spark.sparkContext

Até agora vimos trabalhando com a estrutura base do Spark, as RDDs. Contudo, essas estruturas são trabalhosas e requerem o tratamento manual de tipos de dados bem como a interpretação constante dos dados posicionais na RDD.

É possível simplificar nosso trabalho com o uso de camadas novas e mais elevadas do Spark.


Existe uma série de funções para a criação e a criação e manipulação destas novas estruturas de dados. Vamos começar investigando os DataFrames, inspirados na biblioteca Pandas do Python.


## createDataFrame()
Para criar um DataFrame a partir de uma RDD, podemos utilizar a função `createDataFrame(RDD)`. É importante notar que o ponto de entrada das APIs de dados estruturados não é mais o `sparkContext`, mas diretamente a `SparkSession` acessível a partir da nossa variável `spark`.

In [4]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]



Verificamos que a variável é do tipo DataFrame com colunas com nome `_1, _2,` e diferentes tipos, já __inferidos__ pelo Spark na criação do DataFrame a partir da RDD.

## take()
Podemos então utilizar a função take para verificar nosso DataFrame.

É interessante observar que a função nos mostra uma RDD e não um DataFrame. Essa RDD é composta de objetos do tipo Row, ou então linha. Isso é, um DataFrame nada mais é do que uma RDD onde cada elemento é uma linha de uma tabela com suas diferentes colunas. 

Diferentemente das RDDs padrões, contudo, os objetos Row grava consigo os nomes e os tipos das colunas e o Spark mantém a gestão garantindo que um mesmo DataFrame seja composto de linhas do mesmo tipo.

## Row()

Podemos manualmente criar uma linha, a partir da classe Row.

Para isso precisamos nomear as colunas e seus valores.

Mais tarde entenderemos como concatenar diferentes objetos Row em um DataFrame.

## asDict()

Podemos transformar objetos do tipo Row em dicionários no python. 

Para isso, vamos inicialmente guardar duas linhas de nosso DataFrame.

Na sequencia utilizamos a função asDict()

Com isso podemos acessar os valores de cada coluna normalmente como um dicionário.

## RDD
O atributo rdd do DataFrame também nos dá acesso direto à RDD fundamental que constrói o DataFrame.

Nessa RDD podemos aplicar todas as transformações e ações que aprendemos, desde que respeitando o tipo do elemento Row.

# Acessando o DataFrame
A ideia de termos um DataFrame, contudo, é não precisarmos operar diretamente nas RDDs. Para isso existe um conjunto de funções de acesso direto ao DataFrame. Como estas funções operam intrinsecamente em RDDs, elas também são transformações e ações.


## show()
A ação show() nos permite olhar o DataFrame como uma tabela, muito mais próximo do Pandas DataFrame. Cuidado, contudo, com o desejo de olhar o DataFrame completo.

## printSchema()

A ação show() não nos mostra claramente qual é o tipo dos dados envolvidos (similar a dtypes). Para isso, podemos imprimir o Schema do DataFrame. 

O Schema é a definição do tipo de dado de cada coluna. É importante notar que o Spark inferiu esses dados a partir da RDD. O Spark infere a partir do primeiro elemento nos dados. Portanto, é importante que na leitura de arquivos, caso a inferência seja utilizada, garantir que a primeira linha do arquivo possua o tipo correto.

## dtypes

Podemos também utilizar o atributo dtypes para mostrar o tipo de cada coluna.

# Definindo o Schema
Idealmente, quando operando em grandes massas de dados, a inferência do tipo pela primeira linha não é adequada e utilizamos como boa prática a definição manual do schema. Para isso precisamos utilizar os tipos de dados nativos do Spark que são convertidos para e de tipos python pela API pyspark.

Para isso importamos os tipos do spark.

## StructType(), StructField(), IntegerType(), StringType(), FloatType()
Cada tipo de variável é na verdade um objeto de uma classe. Então acessamos estes objetos para instanciar e para identificar os tipos de cada variável. 

O Schema de um DataFrame é definido como um objeto StructType (similar a uma lista ou tupla) com campos do tipo StructField(). Cada campo (StructField) possui um nome, um tipo (objeto do tipo no spark) e um flag se é permitida a existência de nulos. 

Assim, podemos definir explicitametne um Schema para nosso DataFrame.

Na sequencia, atribuímos o nosso schema ao DataFrame na sua criação.

Podemos então olhar nosso DataFrame e verificar que as colunas agora vêm identificadas.

Podemos também verificar o Schema do DataFrame com as características que definimos anteriormente.

# Operações básicas com colunas

A exemplo do Pandas DataFrame, temos à nossa disposição uma série de transformações e ações que nos permite operar sobre o Spark DataFrame. Veremos a seguir as operações em colunas.

## columns
O atributo columns retorna, a exemplo do Pandas, uma lista com o nome das colunas. Contudo, diferente do Pandas, não é possível sobrescrever este atributo diretamente. 

## withColumnRenamed()
Para renomear uma coluna utilizamos a função withColumnRenamed(). Esta função opera em uma coluna por vez. Assim, podemos utilizar um laço para renomear todas as colunas.

In [ ]:
for old_col, new_col in zip(sdf.columns, ['dept_name', 'dept_id']):
    sdf = sdf.withColumnRenamed(old_col, new_col)

## drop()
Podemos descartar colunas usando a função .drop()

## Acessando colunas
Para acessar as colunas de um DataFrame, podemos utilizar a mesma notação python. Observe, contudo, que as colunas são representadas como objetos e, assim, o acesso à coluna não retorna automaticamente os dados existentes nela.

## select()
Para acessar os dados de uma coluna específica, precisamos utilizar a transformação select() seguida de uma ação show(). Esta transformação é similiar ao SELECT em SQL.

## Case sensitivity
Observe que a exemplo do SQL, DataFrames em Spark não são naturalmente sensíveis ao caso.

# Operações básicas com linhas
Da mesma forma que temos operações com colunas, temos operações com linhas.

## limit()
A transformação limit(LIM) nos permite limitar um número de registros (linhas no DataFrame) a ser retornado. POdemos concatenar com .collect() e temos o DataFrame no formato de RDD. 


Com isso podemos reduzir o número de linhas do nosso DataFrame para aumentar a velocidade do nosso trabalho.

## filter() - filtrando linhas
A transformação filter() nos permite filtrar linhas com base em condições especificadas sobre colunas.

## where() - filtrando linhas

Para manter a similaridade como o SQL, a transformação filter() também pode ser acessada através de seu apelido (alias) where().

## distinct() - filtrando valores distintos
Utilizamos a transformação distinct() para selecionar apenas os valores distintos de uma coluna específica.

## union() - concatenando linhas
Para concatenar linhas no DataFrame, precisamos criar uma RDD com objetos do tipo Row e Schema idêntico.

Transformamos essa lista em uma RDD, usando parallelize(), da mesma forma que fizemos anteriormente.

Por fim, criamos o DataFrame, definindo o Schema.

Podemos então concatenar nosso novo DataFrame ao DataFrame antigo.

## orderBy() - ordenando linhas
Com a transformação orderBy() podemos ordenar as linhas do DataFrame. Para isso, precisamos utilizar os métodos .asc() ou .desc() na coluna chave, indicando a ordem desejada.

# pyspark.sql.functions
Existe uma série de funções que facilitam a manipulação de DataFrames. Elas estão disponíveis no pacote pyspark.sql.functions e podem ser investigadas na referência abaixo:

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

Podemos importá-las todas através da linha de comando abaixo.

In [ ]:
from pyspark.sql import functions as sf

## lit() - criando colunas
Como visto anteriormente, o Spark possui tipos próprios de dados com correspondência com o Python. Se quisermos criar uma coluna constante, podemos evitar o trabalho de criar a coluna em Python e transformá-la, utilizando a função lit(val) que cria uma coluna de literais (constantes) val.

## withColumn() - adicionando colunas

Para isso, precisamos concatenar uma coluna de literais. Isso é feito com a transformação withColumn(nome, nova_col).

## col() - acessando colunas

Em alguns casos precisamos explicitar que desejamos acessar um objeto do tipo Column. Para isso temos à nossa disposição a função .col(nome).

Observe que temos, com isso, diferentes formas de acessar colunas no select. Em alguns casos específicos será necessário optar pela menção explícita ao tipo do objeto usando .col()

Podemos ainda acessar a RDD resultante da transformação.

## alias()

A exemplo do SQL, podemos utilizar o método .alias() de uma coluna específica para renomear colunas rapidamente.

# Expressões
DataFrames são equivalentes de tabelas no Pandas e também de tabelas no SQL. Buscando manter compatibilidade com o SQL, expressões permitem que escrevamos algumas expressões simplificadas de SQL para operar no DataFrame.


## expr()
Para isso, utilizamos a função expr(). Podemos, por exemplo, renomear colunas.

Podemos também operar sobre colunas diretamente, a exemplo do SQL.

## selectExpr()

Como a combinação de .select() e .expr() é muito comum, o spark já nos fornece um atalho: selectExpr().

# Abrindo arquivos CSV
Até agora trabalhamos com DataFrames criados manualmente a partir de uma RDD prévia. Podemos utilizar as funções do pacote .read do Spark para ler diretamente fontes de dados. Para carregar arquivos .csv usamos a função .csv.


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

In [ ]:
df = spark.read.csv('ratings.csv', header=True)

In [ ]:
df

In [ ]:
from pyspark.sql.types import StringType, DoubleType, TimestampType, IntegerType, StructType, StructField

In [ ]:
df = spark.read.csv('../10_dados/movie_lens/ratings.csv', header=True, schema = schema )

In [ ]:
from pyspark.sql.functions import lit